# Cartographie sous python

## 1 La création de carte

Création de cartes  
Le package folium est une interface entre python et la librairie leaflet. Elle s'utilise très facilement, comme nous pouvons le voir ci-dessous pour la création d'une carte zoomée sur la capitale. Les coordonnées indiquées sont la latitude et la longitude de Paris.

In [1]:
#!pip install folium
import folium
paris = folium.Map(location = [48.856578, 2.351828], zoom_start = 12)

Affichons la carte de Paris

In [2]:
paris

## 2 Les marqueurs

La fonction Marker() permet d'ajouter une marque sur la carte pour signifier un emplacement précis. Il est possible de spécifier un contenu à afficher (sous la forme d'un pop-up) lorsque l'utilisateur va cliquer sur le marqueur. Celui-ci sera placé à la localisation fournie en premier, au format [latitude, longitude]. Ce contenu est spécifié dans l'argument popup (ici, une simple chaîne de caractères, mais cela peut être plus élaboré). Il est aussi possible customiser l'icône du marqueur. Une fois ce marqueur créé, il faut l'ajouter à la carte, via la fonction add_to().

Afficher un marqueur à la position [48.856578, 2.351828]

In [3]:
folium.Marker(location=[48.856578, 2.351828], popup='Hôtel de ville').add_to(paris)
paris

## 3 Les cercles

La fonction CircleMarker() permet elle de créer des cercles, en donnant la localisation du centre et le rayon (via le paramètre radius). Idem que pour Marker(), on peut spécifier le contenu de la popup affichée lors d'un clic, et il est nécessaire de l'ajouter à la carte. Il est possible aussi de spécifier la couleur du cercle (via color) et la couleur de remplissage (via fillColor).

Afficher un cercle jaune à la position [48.856578, 2.351828]

In [4]:
folium.CircleMarker(location=[48.856578, 2.351828], color= "yellow", fillColor = "yellow").add_to(paris)
paris

## 4 Les informations géographiques

Pour pouvoir ajouter ces informations, nous allons importer les contours des 20 arrondissements parisiens disponibles en téléchargement sur [cette page](https://opendata.paris.fr/explore/dataset/arrondissements/export/?disjunctive.c_ar&disjunctive.c_arinsee&disjunctive.l_ar&location=13,48.85156,2.32327). Les données à récupérer sont celles au format GeoJSON. 
Pour lire ce fichier, il faudra importer la librairie json

In [9]:
import json
geo = json.load(open("../Data/arrondissements.geojson"))

Vérifions que nous avons bien que nous avons les 20 arrondissements de Paris dans l'objet features, qui est une list.


In [10]:
len(geo["features"])

20

Dans chaque objet de cette liste, nous disposons aussi de différents objets de type Feature. Regardez-les avec la fonction keys()

In [11]:
geo["features"][0].keys()

dict_keys(['type', 'geometry', 'properties'])

Dans le champs properties, il y a différentes informations utiles, dont les coordonnées longitude et latitude du centre.

In [12]:
geo["features"][0]["properties"]

{'n_sq_co': 750001537,
 'perimetre': 6471.5882901,
 'l_ar': '9ème Ardt',
 'surface': 2178303.27487137,
 'geom_x_y': [48.8771635173, 2.33745754348],
 'n_sq_ar': 750000009,
 'l_aroff': 'Opéra',
 'c_arinsee': 75109,
 'c_ar': 9}

## 3 Marqueurs par arrondissements

Avec les informations contenues dans geom_x_y, nous allons pouvoir ajouter un marqueur pour chaque arrondissement, en mettant dans la popup, le nom de l'arrondissement.

In [13]:
paris_arr_marker = folium.Map(location = [48.856578, 2.351828], zoom_start = 12)
for arr in geo["features"]:
    prop = arr["properties"]
    folium.Marker(prop["geom_x_y"], popup = prop["l_ar"]).add_to(paris_arr_marker)
paris_arr_marker

Faites maintenant la même chose avec les cercles pour chacun des arrondissements

In [16]:

for arr in geo["features"]:
    prop = arr["properties"]
    folium.CircleMarker(prop["geom_x_y"], popup = prop["l_ar"], radius = prop["surface"]/1000000).add_to(paris_arr_marker)
paris_arr_marker

## 4 Le choroplète

Pour cela, nous devons devons utiliser la fonction choropleth() de la carte créée. Nous passons dans le paramètre geo_str les données contenues dans le fichier GeoJSON importées plus haut.

In [17]:
#!pip install choropleth-geojson
import choropleth_geojson as cg

ModuleNotFoundError: No module named 'choropleth_geojson'

In [18]:
paris_ch = folium.Map(location = [48.856578, 2.351828], zoom_start = 12)
paris_ch.choropleth(geo_data = geo, fill_color="spectral", line_color="none")
paris_ch

C:\ProgramData\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


## 5 Aller plus loin

Effectuer une cartographie avec une couleur différente pour chaque arrondissement

In [19]:
import pandas

arr_num = pandas.DataFrame({
    "Arrondissement" : [arr["properties"]["l_ar"] for arr in geo["features"]], 
    "Numero" : [arr["properties"]["c_ar"] for arr in geo["features"]]
})
arr_num.head()

,Arrondissement,Numero
0,9ème Ardt,9
1,18ème Ardt,18
2,10ème Ardt,10
3,15ème Ardt,15
4,11ème Ardt,11


In [20]:
paris_ch_arr = folium.Map(location = [48.856578, 2.351828], zoom_start = 12)
paris_ch_arr.choropleth(geo_data = geo, key_on = "feature.properties.l_ar", fill_color = "Set1",
                        data=arr_num, columns=['Arrondissement', 'Numero'])
paris_ch_arr

C:\ProgramData\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [6]:
geo

NameError: name 'geo' is not defined